<a href="https://colab.research.google.com/github/ynonato/Databricks_USTA/blob/main/data_ingestion_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

ruta_carpeta = "/content/drive/MyDrive/secop"
os.makedirs(ruta_carpeta, exist_ok=True)

Instalación de dependencias y conexión a la API

In [3]:
!pip install sodapy

import pandas as pd
from sodapy import Socrata
import time
import hashlib
import sqlite3
from google.colab import userdata

# Obtener los secretos
token = userdata.get('TOKEN_APP')
codigo_dataset = userdata.get('CODIGO_DATASET')

# Cliente autenticado
client = Socrata("www.datos.gov.co", token, timeout=1000)

📥 Descarga paginada de datos

In [4]:
# Parámetros de paginación
limit = 500000
offset = 0
reintentos = 5
lista_DF = []

print(f"Descargando datos del dataset: {codigo_dataset}")

terminado = False  # Controla salida del bucle externo

while not terminado:
    intentos = 0
    while intentos < reintentos:
        try:
            print(f"Descargando lote desde offset {offset}...")
            query = f"""
            SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad,
                   documento_proveedor, estado_del_proceso
            LIMIT {limit} OFFSET {offset}
            """
            results = client.get(codigo_dataset, query=query)

            if not results:
                print("✅ Carga finalizada.")
                terminado = True  # Salir del bucle externo
                break

            df_temp = pd.DataFrame(results)

            # Guardar en carpeta Drive
            ruta_archivo = f"{ruta_carpeta}/data_{offset}.csv"
            df_temp.to_csv(ruta_archivo, index=False)

            lista_DF.append(df_temp)
            print(f"✅ Lote de {len(results)} registros desde offset {offset} guardado en {ruta_archivo}.")
            offset += limit
            break

        except Exception as e:
            intentos += 1
            print(f"⚠️ Error: {e} — Reintentando ({intentos}/{reintentos})...")
            time.sleep(30)
    else:
        print("❌ Máximo número de reintentos alcanzado. Proceso detenido.")
        break


Descargando datos del dataset: rpmr-utcd
Descargando lote desde offset 0...
✅ Lote de 500000 registros desde offset 0 guardado en /content/drive/MyDrive/secop/data_0.csv.
Descargando lote desde offset 500000...
✅ Lote de 500000 registros desde offset 500000 guardado en /content/drive/MyDrive/secop/data_500000.csv.
Descargando lote desde offset 1000000...
✅ Lote de 500000 registros desde offset 1000000 guardado en /content/drive/MyDrive/secop/data_1000000.csv.
Descargando lote desde offset 1500000...
✅ Lote de 500000 registros desde offset 1500000 guardado en /content/drive/MyDrive/secop/data_1500000.csv.
Descargando lote desde offset 2000000...
✅ Lote de 500000 registros desde offset 2000000 guardado en /content/drive/MyDrive/secop/data_2000000.csv.
Descargando lote desde offset 2500000...
✅ Lote de 500000 registros desde offset 2500000 guardado en /content/drive/MyDrive/secop/data_2500000.csv.
Descargando lote desde offset 3000000...
✅ Lote de 500000 registros desde offset 3000000 gua

In [5]:
import pandas as pd
import os

ruta_carpeta = "/content/drive/MyDrive/secop"
lista_DF = []

# Listar todos los archivos CSV en la carpeta
archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]

# Leer cada archivo y agregarlo a la lista
for archivo in archivos:
    ruta_archivo = os.path.join(ruta_carpeta, archivo)
    print(f"Cargando {ruta_archivo} ...")
    df_temp = pd.read_csv(ruta_archivo)
    lista_DF.append(df_temp)

# Concatenar todos los DataFrames
DF_secop = pd.concat(lista_DF, ignore_index=True)
print(f"DataFrame reconstruido con {len(DF_secop)} registros.")


Cargando /content/drive/MyDrive/secop/data_0.csv ...
Cargando /content/drive/MyDrive/secop/data_500000.csv ...
Cargando /content/drive/MyDrive/secop/data_1000000.csv ...
Cargando /content/drive/MyDrive/secop/data_1500000.csv ...
Cargando /content/drive/MyDrive/secop/data_2000000.csv ...
Cargando /content/drive/MyDrive/secop/data_2500000.csv ...
Cargando /content/drive/MyDrive/secop/data_3000000.csv ...
Cargando /content/drive/MyDrive/secop/data_3500000.csv ...
Cargando /content/drive/MyDrive/secop/data_4000000.csv ...
Cargando /content/drive/MyDrive/secop/data_4500000.csv ...
Cargando /content/drive/MyDrive/secop/data_5000000.csv ...
Cargando /content/drive/MyDrive/secop/data_5500000.csv ...
Cargando /content/drive/MyDrive/secop/data_6000000.csv ...
Cargando /content/drive/MyDrive/secop/data_6500000.csv ...
Cargando /content/drive/MyDrive/secop/data_7000000.csv ...
Cargando /content/drive/MyDrive/secop/data_7500000.csv ...
Cargando /content/drive/MyDrive/secop/data_8000000.csv ...
Carg

Generación de ID único (consistencia y control de duplicados)

In [6]:
import hashlib

# Concatenar todas las columnas como string para generar un identificador único
DF_secop['concatenar'] = DF_secop.astype(str).agg('||'.join, axis=1)

# Crear hash SHA256 para generar el ID único
DF_secop['id_unico'] = DF_secop['concatenar'].apply(
    lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest()
)

# Eliminar columna auxiliar (opcional)
DF_secop.drop(columns=['concatenar'], inplace=True)

print(f"✅ Se crearon identificadores únicos para {len(DF_secop):,} registros.")


✅ Se crearon identificadores únicos para 19,549,032 registros.


💾 Almacenamiento organizado en SQLite

In [7]:
import sqlite3

ruta_db = '/content/drive/MyDrive/secop/secop_id.db'
conn = sqlite3.connect(ruta_db)

# Guardar datos limpios
DF_secop.to_sql('secop_id', conn, if_exists='replace', index=False)
print(f"✅ Datos guardados correctamente en {ruta_db}")
conn.close()


✅ Datos guardados correctamente en /content/drive/MyDrive/secop/secop_id.db
